In [1]:
#importing required libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.visualization import plot_histogram
simulator = Aer.get_backend('qasm_simulator')


#function for addition modulo 2^n
def adder(qc,a,b):
    for i in range(1,n):
        qc.cx(b[i],a[i])
    
    for i in reversed(range(1,n-1)):
        qc.cx(b[i],b[i+1])
    
    for i in range(n-1):
        qc.ccx(a[i],b[i],b[i+1])
    
    for i in reversed(range(1,n)):
        qc.cx(b[i],a[i])
        qc.ccx(a[i-1],b[i-1],b[i])
    
    for i in range(1,n-1):
        qc.cx(b[i],b[i+1])
    
    for i in range(n):
        qc.cx(b[i],a[i])
        
#function for rotating a word left
def rotlt(qc,a):
    for i in reversed(range(n-1)):
        qc.cx(a[i+1],a[i])
        qc.cx(a[i],a[i+1])
        qc.cx(a[i+1],a[i]) 
        
#function for the quarter round
def qtrround(qc,a,b):
    adder(qc,a,b)
    for i in range(n):
        qc.cx(a[i],b[i])
    for i in range(2):
        rotlt(qc,b)
        
#function for updating the state 
def stateupdate(qc):
    qtrround(qc,con,iv0)
    qtrround(qc,iv1,key)
    qtrround(qc,key,con)
    qtrround(qc,iv0,iv1)
    
    
#function for the last step addition
def laststep(qc):
    adder(qc,con,conp)
    adder(qc,iv0,iv0p)
    adder(qc,iv1,iv1p)
    adder(qc,key,keyp)
    
#preparing the quantum circuit

n = 3 #the size of each word in the matrix, here we use 3 as the number of qubits available is 32 in IBMQ simulator
con = QuantumRegister(n, name = 'con')
conp = QuantumRegister(n, name = 'conp')
iv0 = QuantumRegister(n, name = 'iv0')
iv0p = QuantumRegister(n, name = 'iv0p')
iv1 = QuantumRegister(n, name = 'iv1')
iv1p = QuantumRegister(n, name = 'iv1p')
key = QuantumRegister(n, name = 'key')
keyp = QuantumRegister(n, name = 'keyp')
cl1 = ClassicalRegister(3)
cl2 = ClassicalRegister(3)
cl3 = ClassicalRegister(3)
cl4 = ClassicalRegister(3)
qc = QuantumCircuit(con,iv0,iv1,key,conp,iv0p,iv1p,keyp,cl1,cl2,cl3,cl4)

#initializing the qubits to the given values
qc.x(iv1[2])
qc.x(iv0[1])
qc.x(con[0])

qc.x(key[0])
qc.x(key[1])
qc.x(key[2])

#storing the initial matrix for the last step addition
for i in range(n):
    qc.cx(con[i],conp[i])
    qc.cx(iv0[i],iv0p[i])
    qc.cx(iv1[i],iv1p[i])
    qc.cx(key[i],keyp[i])
    
stateupdate(qc)
laststep(qc)

for i in range(n):
    qc.measure(con[i],cl1[i])  
for i in range(n):
    qc.measure(iv0[i],cl2[i])
for i in range(n):
    qc.measure(iv1[i],cl3[i])
for i in range(n):
    qc.measure(key[i],cl4[i])

    

job = execute(qc, simulator, shots=1)
result = job.result()
counts = result.get_counts(qc)
#print(counts)
#plot_histogram(counts) 